# 패키지 가져오기

In [53]:
import pandas as pd
import numpy as np
import random
import os

import datetime
from pytz import timezone

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from modules.preprocess import *

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

import xgboost as xgb

# 디렉토리 설정

In [2]:
os.getcwd()

'C:\\Users\\LukeLim\\OneDrive\\바탕 화면\\Projects\\Dacon\\income'

# 데이터 가져오기

In [3]:
train_org = pd.read_csv('./dataset/train.csv')

In [4]:
display(train_org.head().iloc[:, :10])
display(train_org.head().iloc[:, 10:])

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other


,Martial_Status,Household_Status,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,Married,Householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,Separated,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,0
2,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,Divorced,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,850
4,Divorced,Householder,Householder,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570


# One-hot Encoding

In [11]:
oe = OneHotEncoder()

In [176]:
numeric = ['Age']
category = ['Gender', 'Education_Status', 'Employment_Status', 'Industry_Status', 'Occupation_Status',
            'Martial_Status', 'Household_Summary', 'Citizenship', 'Birth_Country', 'Tax_Status', 'Income_Status']
train_num = train_org[numeric]
train_cat = train_org[category]

In [177]:
encoded_train_cat = oe.fit_transform(train_cat)
encoded_train_cat

<20000x138 sparse matrix of type '<class 'numpy.float64'>'
	with 220000 stored elements in Compressed Sparse Row format>

In [178]:
train_df = np.concatenate([encoded_train_cat.toarray(), np.array(train_num)], axis=1)

# Train-Test-Split

In [179]:
X_train, X_test, y_train, y_test = train_test_split(train_df, train_org.iloc[:, -1], train_size=0.8, random_state=777)

In [180]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16000, 139)
(4000, 139)
(16000,)
(4000,)


In [181]:
X_train_org = train_org.iloc[list(y_test.index)].reset_index(drop=True)
zero_income_idx = X_train_org.loc[(X_train_org['Education_Status'] == 'Children') | (X_train_org['Employment_Status'] == 'Not Working') | (X_train_org['Industry_Status'] == 'Not in universe or children') | (X_train_org['Occupation_Status'] == 'Unknown')].index

# Modeling

In [234]:
xgbr = xgb.XGBRegressor(
    n_estimators=150,
    # max_depth=20,
    learning_rate=0.05,
    objective='reg:squarederror',
    booster='gblinear',
    eval_metric='rmse',
    seed=777
)
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=150, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [235]:
# xgbr.feature_importances_

In [236]:
# xgb.plot_importance(xgbr)

In [237]:
predicted = xgbr.predict(X_test)
predicted[zero_income_idx] = 0

In [238]:
np.sqrt(mean_squared_error(predicted, y_test))

570.2684051150541

# Inference

In [239]:
submit_org = pd.read_csv('./dataset/test.csv')

In [240]:
submit_num = submit_org[numeric]
submit_cat = submit_org[category]

In [241]:
encoded_submit_cat = oe.transform(submit_cat)
encoded_submit_cat

<10000x138 sparse matrix of type '<class 'numpy.float64'>'
	with 110000 stored elements in Compressed Sparse Row format>

In [242]:
submit_df = np.concatenate([encoded_submit_cat.toarray(), np.array(submit_num)], axis=1)

In [243]:
submit_zero_income_idx = submit_org.loc[(submit_org['Education_Status'] == 'Children') | (submit_org['Employment_Status'] == 'Not Working') | (submit_org['Industry_Status'] == 'Not in universe or children') | (submit_org['Occupation_Status'] == 'Unknown')].index

In [244]:
predicted = xgbr.predict(submit_df)
predicted[submit_zero_income_idx] = 0

In [245]:
submission = submit_org[['ID']]
submission['Income'] = predicted
submission

,ID,Income
0,TEST_0000,0.000000
1,TEST_0001,0.000000
2,TEST_0002,340.260040
3,TEST_0003,724.808838
4,TEST_0004,0.000000
...,...,...
9995,TEST_9995,891.826904
9996,TEST_9996,656.908081
9997,TEST_9997,411.227722
9998,TEST_9998,0.000000


In [248]:
submission.to_csv('submission/submission_0312_1.csv', index=False)